## Initial setup and imports

In [4]:
import geopandas
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

pd.set_option('display.max_columns', None)

## Initial Data Exploration

In [5]:
element_tree = ET.parse(os.path.join('data', 'ds.xml'))
xml_root = element_tree.getroot()
data_list = []
for establishment in xml_root:
    establistment_dist = {}
    has_inspection_data = False
    for tag in establishment:
        if tag.tag == 'INSPECTION':
            has_inspection_data = True
        else:
            establistment_dist[tag.tag] = tag.text
    if has_inspection_data:
        inspection_dict = {}
        inspections = establishment.findall('INSPECTION')
        for inspection in inspections:
            for tag in inspection:
                if tag.tag != 'INFRACTION':
                    inspection_dict[tag.tag] = tag.text
            data_list.append({**establistment_dist, **inspection_dict})
    else:
        data_list.append(establistment_dist)
dinesafe_df = pd.DataFrame(data_list)

dinesafe_df = dinesafe_df.drop(columns=['ID', 'TYPE']).drop_duplicates()
dinesafe_df

,NAME,ADDRESS,LATITUDE,LONGITUDE,STATUS,DATE
0,'K' STORE,99 CARLTON ST,43.66205,-79.37747,Pass,2019-03-29
1,0109 Dessert + Chocolate,"2190 MCNICOLL AVE, -109",43.81477,-79.29491,Pass,2019-10-25
2,1 PLUS 1 PIZZA,361 OAKWOOD AVE,43.68725,-79.43842,Conditional Pass,2019-03-04
3,1 PLUS 1 PIZZA,361 OAKWOOD AVE,43.68725,-79.43842,Pass,2019-03-08
4,1 PLUS 1 PIZZA,361 OAKWOOD AVE,43.68725,-79.43842,Pass,2019-10-24
...,...,...,...,...,...,...
40923,fimi Kitchens,2958 ISLINGTON AVE,43.75729,-79.57019,Pass,2020-02-03
40924,iQ FOOD CO.,181 BAY ST,43.64748,-79.37849,Pass,2019-06-21
40925,iQ FOOD CO.,181 BAY ST,43.64748,-79.37849,Pass,2019-11-15
40926,iQx,55 AVENUE RD,43.67121,-79.39441,Pass,2020-01-02


In [6]:
address_df = pd.read_csv(os.path.join('data', r'ADDRESS_POINT_WGS84.csv'))
address_df

,GEO_ID,LINK,MAINT_STAG,ADDRESS,LFNAME,LO_NUM,LONUMSUF,HINUM,HINUMSUF,ARC_SIDE,DISTANCE,FCODE,FCODE_DES,CLASS,NAME,X,Y,LONGITUDE,LATITUDE,OBJECTID,MUN_NAME,WARD_NAME,geometry
0,5729533,5729516,REGULAR,404,Lake Promenade,404,NaN,NaN,NaN,R,59.64,100001,Low Density Residential,Land,NaN,301526.896,4827441.250,-79.540536,43.586906,2307121,Etobicoke,Etobicoke-Lakeshore,POINT (-79.54053581499994 43.58690564900007)
1,5729531,5729516,REGULAR,402,Lake Promenade,402,NaN,NaN,NaN,R,44.98,100001,Low Density Residential,Land,NaN,301539.058,4827450.035,-79.540385,43.586985,1771883,Etobicoke,Etobicoke-Lakeshore,POINT (-79.54038525799996 43.58698477500008)
2,5729535,5729516,REGULAR,407,Lake Promenade,407,NaN,NaN,NaN,L,47.00,100001,Low Density Residential,Land,NaN,301576.469,4827413.878,-79.539922,43.586659,1813766,Etobicoke,Etobicoke-Lakeshore,POINT (-79.53992175699995 43.58665947100008)
3,5729534,5729516,REGULAR,405,Lake Promenade,405,NaN,NaN,NaN,L,34.35,100001,Low Density Residential,Land,NaN,301586.074,4827422.263,-79.539803,43.586735,2499509,Etobicoke,Etobicoke-Lakeshore,POINT (-79.53980286499996 43.58673499400004)
4,5729532,5729516,REGULAR,403,Lake Promenade,403,NaN,NaN,NaN,L,20.86,100001,Low Density Residential,Land,NaN,301593.239,4827433.953,-79.539714,43.586840,1818598,Etobicoke,Etobicoke-Lakeshore,POINT (-79.53971420699997 43.58684025600007)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526584,18632,30073953,REGULAR,213,Browning Ave,213,NaN,NaN,NaN,R,19.49,100001,Low Density Residential,Land,NaN,317059.914,4838097.303,-79.347938,43.682722,4081858,East York,Toronto-Danforth,POINT (-79.34793775099996 43.68272237200006)
526585,18634,30073953,REGULAR,215,Browning Ave,215,NaN,NaN,NaN,R,23.96,100001,Low Density Residential,Land,NaN,317064.113,4838098.902,-79.347886,43.682737,4081921,East York,Toronto-Danforth,POINT (-79.34788563099994 43.68273669800004)
526586,31139,30073950,REGULAR,33,Palmer Ave,33,NaN,NaN,NaN,R,51.00,100001,Low Density Residential,Land,NaN,320914.485,4839127.978,-79.300098,43.691926,4081364,East York,Beaches-East York,POINT (-79.30009815599999 43.69192613300004)
526587,31140,30073950,REGULAR,35,Palmer Ave,35,NaN,NaN,NaN,R,56.00,100001,Low Density Residential,Land,NaN,320922.792,4839135.884,-79.299995,43.691997,4081912,East York,Beaches-East York,POINT (-79.29999486299994 43.69199711500005)


In [7]:
# round lat and long remove excess precision
precision = 4
dinesafe_df['LATITUDE'] = dinesafe_df['LATITUDE'].astype('float').round(precision)
dinesafe_df['LONGITUDE'] = dinesafe_df['LONGITUDE'].astype('float').round(precision)

address_df['LATITUDE'] = address_df['LATITUDE'].round(precision)
address_df['LONGITUDE'] = address_df['LONGITUDE'].round(precision)

dine_address_df = dinesafe_df.merge(
    address_df[['LONGITUDE', 'LATITUDE', 'MUN_NAME', 'WARD_NAME']],
    how='left',
    on=['LONGITUDE', 'LATITUDE']
)
dine_address_df = dine_address_df.drop_duplicates()
dine_address_missing_df = dine_address_df[dine_address_df['MUN_NAME'].isnull()]
dine_address_missing_df = dine_address_missing_df.drop(columns=['WARD_NAME', 'MUN_NAME'])
dine_address_df = dine_address_df[~dine_address_df['MUN_NAME'].isnull()]

address_df['LATITUDE'] = address_df['LATITUDE'].round(precision- 1)
address_df['LONGITUDE'] = address_df['LONGITUDE'].round(precision - 1)
dine_address_missing_df = dine_address_missing_df.merge(
    address_df[['LONGITUDE', 'LATITUDE', 'MUN_NAME', 'WARD_NAME']],
    how='left',
    on=['LONGITUDE', 'LATITUDE']
)
dine_address_missing_df = dine_address_missing_df.drop_duplicates()
dine_address_df = pd.concat([dine_address_df, dine_address_missing_df])
dine_address_df

,NAME,ADDRESS,LATITUDE,LONGITUDE,STATUS,DATE,MUN_NAME,WARD_NAME
0,'K' STORE,99 CARLTON ST,43.6620,-79.3775,Pass,2019-03-29,former Toronto,Toronto Centre
2,1 PLUS 1 PIZZA,361 OAKWOOD AVE,43.6872,-79.4384,Conditional Pass,2019-03-04,York,Toronto-St. Paul's
4,1 PLUS 1 PIZZA,361 OAKWOOD AVE,43.6872,-79.4384,Pass,2019-03-08,York,Toronto-St. Paul's
6,1 PLUS 1 PIZZA,361 OAKWOOD AVE,43.6872,-79.4384,Pass,2019-10-24,York,Toronto-St. Paul's
8,1 PLUS 1 PIZZA,361 OAKWOOD AVE,43.6872,-79.4384,Pass,2020-02-14,York,Toronto-St. Paul's
...,...,...,...,...,...,...,...,...
17679,a ! Xiang,3250 MIDLAND AVE,43.8051,-79.2876,Conditional Pass,2019-11-01,NaN,NaN
17680,a ! Xiang,3250 MIDLAND AVE,43.8051,-79.2876,Pass,2019-11-05,NaN,NaN
17681,a ! Xiang,3250 MIDLAND AVE,43.8051,-79.2876,Pass,2020-11-20,NaN,NaN
17682,iQ FOOD CO.,181 BAY ST,43.6475,-79.3785,Pass,2019-06-21,NaN,NaN
